# Load annotated hate speech corpora

## Load Qian+2019 corpus
Has multiple comments in some rows (1,2,3). The ones indexed in ‘hate_speech_idx’ column are the ones annotated for hate

In [ ]:
import pandas as pd
import os

dirpath = '/storage2/mamille3/data/hate_speech/qian2019/'
qian2019 = {}
qian2019['reddit'] = pd.read_csv(os.path.join(dirpath, 'reddit.csv'))
qian2019['reddit']
# qian2019['gab'] = pd.read_csv(os.path.join(dirpath, 'gab.csv'))
# qian2019['gab']

In [ ]:
pd.set_option('display.max_colwidth', None)
# qian2019['gab'].head()[['id', 'text', 'hate_speech_idx']]
qian2019['reddit'].head()[['id', 'text', 'hate_speech_idx']]

In [12]:
# Transform to just extract hateful comments
import re

def select_comment(idx, text):
    match = re.search(r'{}\.(.*)\n'.format(idx), text)
    if match is not None:
         match = match.group(1).strip()
    return match

In [13]:
select_comment('1', '1. something\n')

'something'

## Load Gab Hate Corpus

In [6]:
import pandas as pd
import os

dirpath = '/storage2/mamille3/data/hate_speech/gab_hate_corpus/'
gabhate = pd.read_csv(os.path.join(dirpath, 'GabHateCorpus_annotations.tsv'), sep='\t')
gabhate

,ID,Annotator,Text,Hate,HD,CV,VO,REL,RAE,SXO,GEN,IDL,NAT,POL,MPH,EX,IM
0,27044,4,Ah the PSYOPS antifa crew is back. That’s how ...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,27044,15,Ah the PSYOPS antifa crew is back. That’s how ...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,27044,10,Ah the PSYOPS antifa crew is back. That’s how ...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,27044,8,Ah the PSYOPS antifa crew is back. That’s how ...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,27045,4,Get the new Android app update released today ...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86524,9188,6,"He thinks only peons own guns, you're supposed...",1,1,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
86525,9188,3,"He thinks only peons own guns, you're supposed...",0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86526,9222,11,America must stop funding the United Nations.,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86527,9222,3,America must stop funding the United Nations.,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
len(gabhate.ID.unique())

27665

In [12]:
train = pd.read_csv(os.path.join(dirpath, 'ghc_train.tsv'), sep='\t')
test = pd.read_csv(os.path.join(dirpath, 'ghc_test.tsv'), sep='\t')
ghc = pd.concat([train, test], ignore_index=True)
ghc

,text,hd,cv,vo
0,He most likely converted to islam due to his n...,0,0,0
1,So Ford lied about being a psychologist. Recor...,0,0,0
2,Jobs. Education. Ending abuse of Nation. CA43.,0,0,0
3,"I share a lot of your values, & like many who ...",0,0,0
4,I am so ready to get back to blogging! www.ben...,0,0,0
...,...,...,...,...
27541,"Trump To ""Counter"" DNC Lawsuit; Seeks Servers,...",0,0,0
27542,i guess eu is gonna have to back track a littl...,0,0,0
27543,A good read here.... https://countrysquire....,0,0,0
27544,The only way to change things is to have compa...,0,0,0


In [8]:
ghc['hate'] = ghc.hd | ghc.cv | ghc.vo
ghc

,text,hd,cv,vo,hate
0,He most likely converted to islam due to his n...,0,0,0,0
1,So Ford lied about being a psychologist. Recor...,0,0,0,0
2,Jobs. Education. Ending abuse of Nation. CA43.,0,0,0,0
3,"I share a lot of your values, & like many who ...",0,0,0,0
4,I am so ready to get back to blogging! www.ben...,0,0,0,0
...,...,...,...,...,...
5505,"Trump To ""Counter"" DNC Lawsuit; Seeks Servers,...",0,0,0,0
5506,i guess eu is gonna have to back track a littl...,0,0,0,0
5507,A good read here.... https://countrysquire....,0,0,0,0
5508,The only way to change things is to have compa...,0,0,0,0


In [9]:
ghc.hate.sum()

3377

In [10]:
print(ghc.hd.sum())
print(ghc.cv.sum())
print(ghc.vo.sum())

2348
155
1748


In [13]:
# Format for NetMapper
out = ghc['text']
out.to_csv(os.path.join(dirpath, 'ghc_text.csv'))

## Load de Gibert+2019 Stormfront data

### Files -> CSV

In [14]:
# Load text
from tqdm.notebook import tqdm
from collections import defaultdict

stormfront_dirpath = '/storage2/mamille3/data/hate_speech/degibert2019/'
lines = []
for fname in tqdm(os.listdir(os.path.join(stormfront_dirpath, 'all_files'))):
    file_id = fname.split('.')[0]
    comment_id, sentence_id = file_id.split('_')
    with open(os.path.join(stormfront_dirpath, 'all_files', fname)) as f:
        lines.append([file_id, comment_id, sentence_id, f.read()])
        
len(lines)

  0%|          | 0/10944 [00:00<?, ?it/s]

10944

In [16]:
# Transform to dataframe
import pandas as pd

stormfront_texts = pd.DataFrame(lines, columns=['file_id', 'comment_id', 'sentence_id', 'text'])
stormfront_texts

,file_id,comment_id,sentence_id,text
0,14101013_1,14101013,1,Im looking for a good Aryan white male in or around the central jersey or surrounding area or some new friends from all around
1,30423555_2,30423555,2,"Uzi Tactical Pens Demonstration - YouTube ( I know Uzi is an Israeli company , but the video is pretty good and you can get a Smith and Wesson model. )"
2,13863902_2,13863902,2,They speak the Irish language because it 's their culture but are brainwashed into thinking that it 's ok to destroy our race .
3,32923728_3,32923728,3,And now this clip suggests they were suffering by having to live among heartless and barbarian Whites .
4,14270200_2,14270200,2,"Missouri is very cheap I have lived her like I said most of my life you can get 130 acres , with a 4 bedroom house , barns , ponds and woods for 97,000 in southern Missouri ."
...,...,...,...,...
10939,32625866_1,32625866,1,"I found it strange the way the police handled this case , they told her father and his friends to search the house for her , all those people wandering around the house contaminating all the evidence , that 's hardly good police procedure ."
10940,30386283_2,30386283,2,The sexual references are there .
10941,30687903_2,30687903,2,I hope people can pass out candy while watching that game .
10942,14668109_1,14668109,1,Not to mention that its a business in jails among the gangs too .


In [11]:
# Add in hate labels
annotations = pd.read_csv(os.path.join(stormfront_dirpath, 'annotations_metadata.csv'))
annotations

,file_id,user_id,subforum_id,num_contexts,label
0,12834217_1,572066,1346,0,noHate
1,12834217_2,572066,1346,0,noHate
2,12834217_3,572066,1346,0,noHate
3,12834217_4,572066,1346,0,hate
4,12834217_5,572066,1346,0,noHate
...,...,...,...,...,...
10939,33676864_5,734541,1388,0,noHate
10940,33677019_1,735154,1388,0,noHate
10941,33677019_2,735154,1388,0,noHate
10942,33677053_1,572266,1388,0,hate


In [21]:
stormfront_data = pd.merge(stormfront_texts, annotations, on=['file_id']).set_index('file_id')
stormfront_data

,comment_id,sentence_id,text,user_id,subforum_id,num_contexts,label
file_id,,,,,,,
14101013_1,14101013,1,Im looking for a good Aryan white male in or around the central jersey or surrounding area or some new friends from all around,599276,1381,0,noHate
30423555_2,30423555,2,"Uzi Tactical Pens Demonstration - YouTube ( I know Uzi is an Israeli company , but the video is pretty good and you can get a Smith and Wesson model. )",572231,1359,0,noHate
13863902_2,13863902,2,They speak the Irish language because it 's their culture but are brainwashed into thinking that it 's ok to destroy our race .,574905,1391,1,noHate
32923728_3,32923728,3,And now this clip suggests they were suffering by having to live among heartless and barbarian Whites .,575163,1345,0,hate
14270200_2,14270200,2,"Missouri is very cheap I have lived her like I said most of my life you can get 130 acres , with a 4 bedroom house , barns , ponds and woods for 97,000 in southern Missouri .",599136,1380,0,noHate
...,...,...,...,...,...,...,...
32625866_1,32625866,1,"I found it strange the way the police handled this case , they told her father and his friends to search the house for her , all those people wandering around the house contaminating all the evidence , that 's hardly good police procedure .",577942,1345,0,noHate
30386283_2,30386283,2,The sexual references are there .,572804,1362,0,noHate
30687903_2,30687903,2,I hope people can pass out candy while watching that game .,578188,1348,0,noHate


In [22]:
stormfront_data.to_csv(os.path.join(stormfront_dirpath, 'combined_data.csv'))

### Format texts for DocuScope
Grouping by author, etc. Found out that this is unnecessary. I just ran the individual sentence files through DocuScope

In [2]:
# Load annotations
import pandas as pd

annotations_fpath = '/storage2/mamille3/data/hate_speech/degibert2019/combined_data.csv'
annotations = pd.read_csv(annotations_fpath).sort_values(['comment_id', 'sentence_id']).set_index('file_id')
annotations

,comment_id,sentence_id,text,user_id,subforum_id,num_contexts,label
file_id,,,,,,,
12834217_1,12834217,1,"As of March 13th , 2014 , the booklet had been...",572066,1346,0,noHate
12834217_2,12834217,2,In order to help increase the booklets downloa...,572066,1346,0,noHate
12834217_3,12834217,3,( Simply copy and paste the following text int...,572066,1346,0,noHate
12834217_4,12834217,4,Click below for a FREE download of a colorfull...,572066,1346,0,hate
12834217_5,12834217,5,Click on the `` DOWNLOAD ( 7.42 MB ) '' green ...,572066,1346,0,noHate
...,...,...,...,...,...,...,...
33677015_1,33677015,1,Apparently he came to the conclusion that his ...,572948,1388,0,noHate
33677019_1,33677019,1,Wish we at least had a Marine Le Pen to vote f...,735154,1388,0,noHate
33677019_2,33677019,2,Its like the choices are white genocide candid...,735154,1388,0,noHate


In [53]:
# Group by comment
stormfront_data['sentence_id'] = stormfront_data['sentence_id'].astype(int)
# gped = stormfront_data.sort_values(['comment_id', 'sentence_id']).groupby('comment_id').agg({'text': ' '.join, 'sentence_id': lambda x: ','.join(x.astype(str))})
comment_gps = stormfront_data.sort_values(['comment_id', 'sentence_id']).groupby('comment_id').agg(
    {
     'sentence_id': lambda x: list(x.astype(str)),
        'user_id': 'first',
        # 'user_id': list,
        'text': ' '.join, 
     'label': list,
    })
comment_gps

,sentence_id,user_id,text,label
comment_id,,,,
12834217,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",572066,"As of March 13th , 2014 , the booklet had been downloaded over 18,300 times and counting . In order to help increase the booklets downloads , it would be great if all Stormfronters who had YouTube accounts , could display the following text in the description boxes of their uploaded YouTube videos . ( Simply copy and paste the following text into your YouTube videos description boxes. ) Click below for a FREE download of a colorfully illustrated 132 page e-book on the Zionist-engineered INTENTIONAL destruction of Western civilization . Click on the `` DOWNLOAD ( 7.42 MB ) '' green banner link . Booklet updated on Feb. 14th , 2014 . ( Now with over 18,300 Downloads. ) PDF file : http://www.mediafire.com/download/7p3p3goadvvqvsf/WNDebateBooklet_2-14-14.pdfMSWordfile:http://www.mediafire.com/download/psezkkk4d6a3wt1/WNDebateBooklet _ 2-14-14.docx Watch the 10 hour video version of `` The Zionist Attack on Western Civilization '' @ http://trutube.tv/video/14247/The-Zionist-Attack-on-Western-Civilization-Pages-1-33-Part-1-of-4-Banned-from-YouTubeNotepadPromotionalYouTubeComment:http://www.mediafire.com/download/96fg6ft02lyfruz/Booklet _ White _ YT _ Comment _ ( Hyperlinked ) .txt http://www.mediafire.com/download/zcn3wozjbwnezms/Booklet-White-YT-Comment- ( Hyperlinked-Back-Up % 5D.txt http://www.mediafire.com/download/9uyudq1yuxu1dur/Booklet+Comment+%28Firefox%29.txt2minutepromotionalBOOKLETvideo@http://www.youtube.com/watch ? v = _ 8hg254ALpM Are you interested in helping spread the booklet download link across the world ? Then why not simply copy this text ( & links ) and paste it into the description box of your YouTube videos ? Thank you in advance. : ) Download the youtube `` description box '' info text file below @ http://www.mediafire.com/download/dqhn1czprr17o21/Booklet-Description-Box _ Info.txt","[noHate, noHate, noHate, hate, noHate, noHate, noHate, noHate, noHate, noHate]"
12834278,"[1, 2, 3, 4, 5, 6, 7, 8, 9]",572066,"As of December 31st , 2013 at about 8 AM PST , the booklet had been downloaded 16,270 times and counting . In order to help increase the booklets downloads , it would be great if all Stormfronters who had YouTube accounts , could display the following text in the description boxes of their uploaded YouTube videos . Click below for a FREE download of a colorfully illustrated 132 page e-book on the Zionist-engineered INTENTIONAL destruction of Western civilization . Click on the `` DOWNLOAD ( 7.39 MB ) '' link . Booklet updated on Dec. 9th , 2013 . ( Now over 16,000 Downloads ! ) PDF file : http://www.mediafire.com/download/4w18ocon384cx6c/WN-Debate-Booklet+12-09-13.pdfMSWordfile:http://www.mediafire.com/download/7ic1n7oq7jidz22/WN-Debate-Booklet + 12-09-13.docx Watch the 10 hour video version of `` The Zionist Attack on Western Civilization '' @ http://trutube.tv/video/14247/The-Zionist-Attack-on-Western-Civilization-Pages-1-33-Part-1-of-4-Banned-from-YouTubeNotepadPromotionalYouTubeComment:http://www.mediafire.com/download/96fg6ft02lyfruz/Booklet _ White _ YT _ Comment _ ( Hyperlinked ) .txt http://www.mediafire.com/download/zcn3wozjbwnezms/Booklet-White-YT-Comment- ( Hyperlinked-Back-Up % 5D.txt http://www.mediafire.com/view/9uyudq1yuxu1dur/Booklet+Comment+%28Firefox%29.txtOriginal2minutepromotionalBOOKLETvideo@http://www.youtube.com/watch ? v = n663eVTSyS8 Are you interested in helping spread the booklet download link across the world ? Then why not simply copy this text ( & links ) and paste it into the description box of your YouTube videos ? Thank you in advance. : ) Download the youtube `` description box '' info text file below @ http://www.mediafire.com/download/gxv2b33v37la6pz/Booklet % 20Description % 20Box % 20Info.txt","[noHate, noHate, hate, noHate, noHate, noHate, noHate, noHate, noHate]"
12834493,[1],572058,"She may or may not be a Jew but she 's certainly stupid , she seems to think the Black

In [54]:
# Group by author
author_gps = comment_gps.sort_index().groupby('user_id').agg({
     'sentence_id': list,
        'text': '\n\n'.join, 
     'label': list,
})
author_gps

,sentence_id,text,label
user_id,,,
572033,"[[1, 2], [1, 2]]","Emporion ballista in Museu d & #39 ; Arqueologia de Catalunya , Barcelona , Spain - YouTube Remains of the ballista found from archaeological excavations in Greek town of Ἐμπόριον ( Emporion ) near present day town of L' Escala in Catalonia , Spain . Further pictures and details from `` Greek and Roman Artillery Wiki '' .\n\nThe only type of museum a zoo needs , is a museum about animals and nature . Holocaust has no business in there .","[[noHate, noHate], [noHate, hate]]"
572036,[[1]],"I'm just happy they have n't tried to cast blacks , Asians , or some other races like Hollywood 's treatment of Thor .",[[noHate]]
572037,"[[1], [1], [1]]",put the kids in a ophanage and arrest the whore of a mother to a point of not having more biracial kids 8 kids have to suffer all their lives to due a whore\n\ni am a white nationalist of a christian faith but still am a white nationalist for racial survival the anti racist christians are the true chirstians ememy\n\nWhites wo n't exist in Canada anymore they all have left or mixed themselves away basically leaving Canada to become a large ghetto,"[[hate], [hate], [hate]]"
572041,"[[1], [1]]","I'd say that here on SF , most of the thread derailers are crypto-antis who do n't give hoot about Thor or Jesus .\n\nAnd it works in our favor the other way too - pushing White people too far so as to cause them to rebel .","[[noHate], [noHate]]"
572042,"[[1], [1, 2], [1], [1, 2, 3, 4], [1, 2, 3, 4], [1]]","The reason large white families were encouraged those days was to build up the population of white Germans , which were ravaged by the jewish attacks at them during WW1 and in hyperinflation 1920 's .\n\nIt used to be the churches in your own community would take care of people facing hard times ... The churches also used to operate hospitals .\n\nIt looks like I have to create MY own copy of PDF file , since the OP version is a read-only item. ( sigh )\n\nleft: Satzvey Castle . The pearl of German Water Castles ............. , , , , , , , , , , , ... right : Burg Eltz Castle is a medieval castle nestled in the hills above the Moselle River between Koblenz and Trier , Germany . It is still owned by a branch of the same family that lived there in the 12th century , 33 generations ago . The Rübenach and Rodendorf families. .\n\nYes, you 've been talking that way aloud in the public for most of your life . If you really want to change that bad grammar habit , simply practice at home ( or in your work cubicle ) - say out loud `` would have '' , `` should have '' , `` could have '' 100 times a day for each phrase ( yes for a total of 300 times per day ) until it is part of your normal everyday speech . And encourage your friends/acquaintances you see daily to change this bad habit . Hope this helps. .\n\nThat was on a sheet of paper , and I take it you meant bull sheet ( as other poster said it before )","[[noHate], [noHate, noHate], [noHate], [noHate, noHate, noHate, noHate], [noHate, noHate, noHate, noHate], [noHate]]"
...,...,...,...
758788,"[[1, 2], [1, 2]]","I think there might be one or two groups if they are still active I have no idea Maybe check the Daily Stormer Book Clubs ? Ive never done it but heard its a great way to meet people ... although Anglin is n't my favoriate `` white nationalist '' ...\n\nI would never join a Christian group , or a group that would allow Christians to join . So my interests in this group are minimal .","[[noHate, noHate], [noHate, noHate]]"
760015,[[1]],"I joined today and just found this sight recenty , I Love being white and wanted to talk to others with veiws simular to my own .",[[noHate]]
775309,"[[1, 2]]",We can turn the cold useless parts of Canada into a dumping ground for Jews and arm the Indians to keep them in line . Toronto and Vancouver will be returned to the white Canadians and join America as the 51st and 52nd states .,"[[hate, noHate]]"


In [55]:
# Get average length of posts
author_gps['num_words'] = [len(x) for x in author_gps.text.str.split()]
author_gps.num_words.mean()

70.40508595988538

In [39]:
gped.num_words.mean() # for comments

39.3142

In [57]:
# Save out documents in individual text files
out_dirpath = os.path.join(stormfront_dirpath, 'grouped_by_author')
for user_id, text in zip(author_gps.index, author_gps.text):
    outpath = os.path.join(out_dirpath, f'{user_id}.txt')
    with open(outpath, 'w') as f:
        f.write(text)

0it [00:00, ?it/s]

## Load ElSherief+2021 
Tweets from hate groups labeled with implicit/explicit hate speech

In [1]:
import pandas as pd

data = pd.read_csv('/storage2/mamille3/data/hate_speech/elsherief2021/implicit_hate_v1_stg1_posts.tsv', sep='\t')
data

,post,class
0,""" : jewish harvard professor noel ignatiev w...",implicit_hate
1,b.higher education is a part of european cult...,not_hate
2,"has a problem with "" the whites "" "" and "" "" ...",not_hate
3,is yasir qadhi a hate preacher for calling ch...,not_hate
4,"rt "" : how three million germans mass murder...",not_hate
...,...,...
21475,"Seeing prostitutes is morally wrong, but being...",implicit_hate
21476,I wonder how many females I raped today I spen...,implicit_hate
21477,Having a criminal record is more attractive to...,implicit_hate
21478,Another advantage of being a female: getting b...,implicit_hate


### Format texts for DocuScope
Save out as individual text files

In [6]:
import os
from tqdm.notebook import tqdm

for text_id, text in tqdm(zip(data.index, data.post), total=len(data)):
    with open(os.path.join('/storage2/mamille3/data/hate_speech/elsherief2021/texts/', f'{text_id}.txt'), 'w') as f:
        f.write(text)

  0%|          | 0/21480 [00:00<?, ?it/s]